Download the date-filtered (e.g.: whole year) from https://www.sbb.ch/de/kaufen/pages/bestellung/bestellungen.xhtml and safe it somewhere.

- pip install pdfminer.six
- Create a file "name.txt" in this folder containing your Name and Surname without quotes ´Mara Muster´.

In [17]:
# Replace this with the path to your downloaded PDF file
pdf_file = "tickets.pdf"

# If your downloaded File is in German, the following, as well as the regex later on should be fine, otherwise you need to make some adjustments
split_words = ['Reisende:r:', 'Bestellübersicht', 'Seite']

# Loads your name from the "name.txt" file --> for privacy and not to include tickets you bought for others.
with open('name.txt', 'r') as f:
    name = f.read()

In [81]:
from pdfminer.high_level import extract_text
import re
text = extract_text(pdf_file)

In [80]:
parts = text.split(split_words[0])
parts = [part for part in parts if part != []] # Remove empty elements from the list

for i, part in enumerate(parts):
    for word in split_words:
        if word in part:
            parts[i] = part.split(word)[1]

# Drop every line that does not start with the name
parts = [part for part in parts if part.startswith(' ' + name)]


In [ ]:
pattern = rf'{name}(?P<ticket_type>.+?)(?P<price>\d+\.\d{{2}} CHF)Reisedatum: (?P<reisedatum>\d{{2}}\.\d{{2}}\.\d{{4}})Lieferadresse: (?P<lieferadresse>[^B]+)Bestelldatum: (?P<bestelldatum>\d{{2}}\.\d{{2}}\.\d{{4}})Bestellnummer: (?P<bestellnummer>\d+)'

result_list = []

for item in parts:
    match = re.search(pattern, item)
    if match:
        result_list.append({
            "Name": name, 
            "Ticket Type": match.group("ticket_type").strip(),
            "Price": match.group("price").strip(),
            "Reisedatum": match.group("reisedatum").strip(),
            "Lieferadresse": match.group("lieferadresse").strip(),
            "Bestelldatum": match.group("bestelldatum").strip(),
            "Bestellnummer": match.group("bestellnummer").strip(),
        })


In [89]:
import pandas as pd

df = pd.DataFrame(result_list)

df.to_csv('tickets.csv')
